In [ ]:
from datasets import load_dataset
from googletrans import Translator, LANGUAGES
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AutoConfig, T5Tokenizer, T5ForConditionalGeneration
from summarizer import Summarizer
import pandas as pd

# Код инференса
https://libraries.io/pypi/bert-extractive-summarizer#examples

In [ ]:
# Создаем объект переводчика
translator = Translator()

# EXT
# Объявляем модель, токенизатор, конфиг
ext_model_name = 'DeepPavlov/rubert-base-cased' # берем русский берт как основу суммаризатора
custom_tokenizer = BertTokenizer.from_pretrained(ext_model_name)
custom_config = AutoConfig.from_pretrained(ext_model_name)
# вывод скрытого состояния каждого слова = True. Это нужно, т.к. Summarizer работает именно с скрытыми состояниями (метаданными о тексте)
custom_config.output_hidden_states=True
custom_model = BertForSequenceClassification.from_pretrained(ext_model_name, config=custom_config)
ext_model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer) # задаём русский берт как модель и токенизатор у суммаризатора

# ABS
# Объявляем модель и токенизатор
abs_model_name = "sarahai/ruT5-base-summarizer"
abs_tokenizer = T5Tokenizer.from_pretrained(abs_model_name)
abs_model = T5ForConditionalGeneration.from_pretrained(abs_model_name)

/opt/conda/envs/interns_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def get_abstract_summary(text, min_length=50, max_length=150):
    ''' Функция создания реферата моделью ruT5'''
    input_ids = abs_tokenizer(text, return_tensors="pt").input_ids # превращаем текст в токены

    outputs = abs_model.generate(
        input_ids, # токены последовательности
        min_length=min_length, # ограничение на минимум слов
        max_length=max_length, # ограничение на максимум слов
        num_beams=4, # beam search, кол-во параллельно-генерируемых последовательностей (а затем выбор одной, наиболее репрезентабельной)
        early_stopping=True # ранняя остановка
    )

    summary = abs_tokenizer.decode(outputs[0], skip_special_tokens=True) # превращаем токены обратно в текст
    return summary

In [ ]:
# загружаем данные
data = pd.read_pickle('ext_data.pkl')
data.head()

,src,labels,ru_src
2965,[Armed men killed eight people in a raid on a ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[В субботу вечером вооруженные люди убили восе...
7792,"[The son of the Sultan of Brunei, one of the w...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Сын султана Брунея, одного из самых богатых л..."
4540,"[Creating romantic, sepia-toned images is a ci...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[Теперь, когда у нас есть фильтры Instagram, с..."
2138,[Manchester City midfielder Yaya Toure is a dr...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",[По словам генерального директора Марко Фассон...
932,"[(CNN)Editor's Note (July 8, 2015) -- Question...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[(CNN) Примечание редактора (8 июля 2015 г.) —...


In [ ]:
test_text = ' '.join(data['ru_src'].iloc[0])
test_text

'В субботу вечером вооруженные люди убили восемь человек во время рейда на футбольный фан-клуб Сан-Паулу, который имеет тесные связи с печально известной, ныне закрытой тюрьмой. Полиция сообщила, что свидетели рассказали им, что около полуночи в субботу трое боевиков ворвались в темное и песчаное место, где собирается фан-группа "Коринтианс" "Павильяо 9", под эстакадой шоссе. Они приказали жертвам лечь лицом вниз на землю и выстрелили семерым в головы. Восьмой мужчина был застрелен, но попытался сбежать, добрался до заправочной станции, а затем был доставлен в ближайшую больницу, где скончался. Вооруженная полиция охраняет штаб-квартиру фан-клуба «Коринтианс»\xa0Pavilhao 9 в воскресенье утром. Обезумевшая женщина разговаривает по мобильному телефону возле здания, где были казнены семеро мужчин. Полицейский показывает гильзы, собранные на месте стрельбы к западу от Сан-Паулу. Детектив Арлиндо Хосе Неграо исключил драку между конкурирующими группировками фанатов. Однако он не стал раскры

In [ ]:
len(data['labels'].iloc[0])

28

In [ ]:
%%time
ext_model(
    test_text,
    ratio=0.2,
    # num_sentences=5
)

CPU times: user 761 ms, sys: 124 ms, total: 885 ms
Wall time: 873 ms


'В субботу вечером вооруженные люди убили восемь человек во время рейда на футбольный фан-клуб Сан-Паулу, который имеет тесные связи с печально известной, ныне закрытой тюрьмой. Полиция сообщила, что свидетели рассказали им, что около полуночи в субботу трое боевиков ворвались в темное и песчаное место, где собирается фан-группа "Коринтианс" "Павильяо 9", под эстакадой шоссе. Однако он не стал раскрывать мотивы нападения, заявив, что это может помешать расследованию. « Пострадавшие в это время готовили барбекю в клубе и мастерили флаги в цветах своей команды для ожидаемого в воскресенье матча между «Коринтианс» и «Палмейрас». Полицейский утешает женщину, которая разговаривает по мобильному телефону возле фан-клуба. Возмущенные игрой команды, они напали на сотрудников и схватили перуанского нападающего Паоло Герреро за шею, вынудив других игроков бежать в раздевалку и забаррикадироваться там до прибытия полиции.'

In [ ]:
%%time
get_abstract_summary(
    text=test_text,
    min_length=10,
    max_length=20,
)

CPU times: user 19 s, sys: 1.34 s, total: 20.4 s
Wall time: 2.93 s


'Полиция Бразилии расследует убийство восьми человек в фан-клубе «Коринтианс'

# Код перевода

In [ ]:
dataset = load_dataset('eReverter/cnn_dailymail_extractive', split='test')
data = dataset.to_pandas().drop('tgt', axis=1).sample(1000)
data

,src,labels
2965,[Armed men killed eight people in a raid on a ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
7792,"[The son of the Sultan of Brunei, one of the w...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4540,"[Creating romantic, sepia-toned images is a ci...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."
2138,[Manchester City midfielder Yaya Toure is a dr...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
932,"[(CNN)Editor's Note (July 8, 2015) -- Question...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...
3582,[Lewis Hamilton remains the richest sportsman ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1081,"[(CNN)China's cybercensors have long used a ""G...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2354,[Graeme McDowell insisted he would never consi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
9202,[The pursuit of journalists for paying public ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [ ]:
data['src'] = data['src'].apply(lambda text: text.tolist())
data['ru_src'] = data['src'].apply(lambda sentences: [translator.translate(sentence, src='en', dest='ru').text for sentence in sentences])
data

In [ ]:
data.to_pickle('ext_data.pkl')